Creating Decorator that can take an argument 

In [11]:
def decoratorfxn(func1 , a:'Extra Parameter'):
    def inner():
        print("Execution started")
        sum = a + 10 # a is free variable here
        func1()
        print("Executed ! ")
        print("sum : {0}".format(sum))
    return inner

def my_function():
    print("My Function executed ") 
    
my_function = decoratorfxn(my_function , 5)  
my_function() # calling



        

Execution started
My Function executed 
Executed ! 
sum : 15


But it dont work with @decoratorfxn(10)So rethinking solution

You can do like this

In [22]:
# using Nested closures to solve this problem
def outer(a):
    def decoratorfxn(func1):
        def inner():
            print("Execution started")
            sum = a + 10 # a is free variable here
            func1()
            print("Executed ! ")
            print("sum : {0}".format(sum))
        return inner
    return decoratorfxn


In [23]:
#dec = outer(10)(my_function)

@outer(10)
def my_function():
    print("My Function executed ") 


In [25]:
my_function()

Execution started
My Function executed 
Executed ! 
sum : 20


### Decorator Application (Decorator Class)

In [26]:
#parameterized decorator
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print('decorated function called: a={0}, b={1}'.format(a, b))
            return fn(*args, **kwargs)
        return inner
    return dec

In [27]:
@my_dec(10, 20)
def my_func(s):
    print('hello {0}'.format(s))

In [28]:
my_func('world')

decorated function called: a=10, b=20
hello world


In [29]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self):
        print('MyClass instance called: a={0}, b={1}'.format(self.a, self.b))

In [30]:
my_class = MyClass(10, 20)

In [31]:
my_class()

MyClass instance called: a=10, b=20


####So let's modify this just a bit, and have the __call__ method be our decorator!

In [32]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print('MyClass instance called: a={0}, b={1}'.format(self.a, self.b))
            return fn(*args, **kwargs)
        return inner

In [33]:
@MyClass(10, 20)
def my_func(s):
    print('Hello {0}!'.format(s))

In [35]:
my_func("Hello World")

MyClass instance called: a=10, b=20
Hello Hello World!


Remember that @MyClass(10, 20) returned an object of type MyClass. But that object is itself callable, so we could do something like:

my_func = MyClass(10, 20)(my_func)

or, more simply

@MyClass(10, 20) def my_func(s): print(s)

## Decorator Application: Decorating Classes



In [36]:
from datetime import datetime, timezone
def debug_info(cls):
    def info(self):
        results = []
        results.append('time: {0}'.format(datetime.now(timezone.utc)))
        results.append('class: {0}'.format(self.__class__.__name__))
        results.append('id: {0}'.format(hex(id(self))))
        
        if vars(self):
            for k, v in vars(self).items():
                results.append('{0}: {1}'.format(k, v))
        
        # we have not covered lists, the extend method and generators,
        # but note that a more Pythonic way to do this would be:
        #if vars(self):
        #    results.extend('{0}: {1}'.format(k, v) 
        #                   for k, v in vars(self).items())
        
        return results
    
    cls.debug = info
    
    return cls

In [37]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return 'Hello there!'

In [38]:
p1 = Person('John', 1939)

In [39]:
p1.debug()

['time: 2023-03-05 14:57:15.003242+00:00',
 'class: Person',
 'id: 0x1c2f9d44c88',
 'name: John',
 'birth_year: 1939']

## Decorator Application: Single Dispatch Generic Functions

In [40]:
from html import escape

def html_escape(arg):
    return escape(str(arg))
                      
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))
                                  
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')
                                  
def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item)) 
             for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
                                  
def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), html_escape(v)) 
             for k, v in d.items())    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [41]:
print(html_str("""this is 
a multi line string
with special characters: 10 < 100"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &lt; 100
